In [1]:
import pandas as pd
import numpy as np
import collections

from scipy.stats import poisson
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score, mean_squared_error

In [2]:
matches = pd.read_csv('teams_matches_stats-2.csv')

## Бейзлайн

In [306]:
## Считаем аггрегированные показатели по забитым и пропущенным голам для выездных и домашних матчей
agg_stats_matches_home = matches_train[
  matches_train['venue'] == 'Home'
].groupby('team')[['GA', 'GF']].mean()
agg_stats_matches_away = matches_train[
  matches_train['venue'] == 'Away'
].groupby('team')[['GA', 'GF']].mean()

In [307]:
def predict_points(agg_stats_home, agg_stats_away, home, away):
    ## В случае отсутствия информации о показателях для команды заполняем все нулями
    if home not in agg_stats_home.index or home not in agg_stats_away.index:
        lamb_home = 0
    else:
        lamb_home = agg_stats_home.at[home,'GF'] * agg_stats_away.at[home,'GA']
    if away not in agg_stats_away.index or away not in agg_stats_home.index:
        lamb_away = 0
    else:
        lamb_away = agg_stats_away.at[away,'GF'] * agg_stats_home.at[away,'GA']

    ## Проходимся по всем возможным результатам матчей
    prob_home, prob_away, prob_draw = 0, 0, 0
    for x in range(0,11): #количество голов команды хозяев
        for y in range(0, 11): #количество голов команды гостей
            p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
            if x == y:
                prob_draw += p
            elif x > y:
                prob_home += p
            else:
                prob_away += p
    ## Считаем предполагаемые количества очков
    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    return (points_home, points_away)

In [308]:
## Демонстрационный пример
## Первое значение -- ожидаемое количество очков Ajaccio, второе -- Alavés
predict_points(agg_stats_matches_home, agg_stats_matches_away, 'Ajaccio', 'Alavés')

(0.34993774911115544, 2.3001244525162505)

In [309]:
## Функция определения победителя с помощью предсказания
def get_winner_preds(agg_stats_matches_home, agg_stats_matches_away, home, away):
    points_home, points_away = predict_points(
        agg_stats_matches_home,
        agg_stats_matches_away,
        home, away
        )
    if points_home > points_away:
        return home
    return away

## Фунция определения реального победителя
def get_winner(x):
    if x['GA'] < x['GF']:
        return x['team']
    return x['opponent']

## Функция перекодирования переменных в бинарные
## Таргет 1 -- победа или ничья домашней команды
def recode(x, winner):
    if x['team'] == x[winner] and x['venue'] == 'Home':
        return 1
    return 0

In [310]:
## Получаем реального победителя матча и предсказание модели для тренировочных данных
matches_train['winners_train'] = matches_train.apply(lambda x: get_winner(x), axis=1)
matches_train['winners_train_preds'] = matches_train.apply(
    lambda x: get_winner_preds(agg_stats_matches_home, agg_stats_matches_away, x['team'], x['opponent']), axis=1
    )

<ipython-input-310-f42de2582836>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_train['winners_train'] = matches_train.apply(lambda x: get_winner(x), axis=1)
<ipython-input-310-f42de2582836>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_train['winners_train_preds'] = matches_train.apply(


In [311]:
## Переводим таргет в бинарный флаг и считаем метрики для тренировочных данных
winners_bin_train = matches_train.apply(lambda x: recode(x, 'winners_train'), axis=1)
winners_bin_train_preds = matches_train.apply(lambda x: recode(x, 'winners_train_preds'), axis=1)
print(f'f1-score на тренировочной выборке: {f1_score(winners_bin_train, winners_bin_train_preds)}')

f1-score на тренировочной выборке: 0.6785241248817407


In [312]:
## Получаем реального победителя матча и предсказание модели для тестовых данных
matches_val['winners_test'] = matches_val.apply(lambda x: get_winner(x), axis=1)
matches_val['winners_test_preds'] = matches_val.apply(
    lambda x: get_winner_preds(agg_stats_matches_home, agg_stats_matches_away, x['team'], x['opponent']), axis=1
    )

<ipython-input-312-d27d04c0e2a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_val['winners_test'] = matches_val.apply(lambda x: get_winner(x), axis=1)
<ipython-input-312-d27d04c0e2a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_val['winners_test_preds'] = matches_val.apply(


In [313]:
## Переводим таргет в бинарный флаг и считаем метрики для тестовых данных
winners_bin_test = matches_val.apply(lambda x: recode(x, 'winners_test'), axis=1)
winners_bin_test_preds = matches_val.apply(lambda x: recode(x, 'winners_test_preds'), axis=1)
print(f'f1-score на тестовой выборке: {f1_score(winners_bin_test, winners_bin_test_preds)}')

f1-score на тестовой выборке: 0.6998841251448435
